In [1]:
# Importing packages...
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns

In [2]:
# Reading data..
ebay_data = pd.read_csv('EbayPcLaptopDataUnclean.csv')
ebay_ID = pd.read_csv('EbayPcLaptopPriceData.csv')

# merging ebay_ID with ebay_data to give every entry a price figure
ebay_data = pd.merge(ebay_data, ebay_ID, on='Item Number', how='left')

# observe what the structure of our data looks like. to make use of this dataset, we will try to find some columns that can be taken at numeric values.
# first, let's look at a random set of observations to see an example of what we are working with
ebay_data.loc[700:709]

,Item Number,Brand,Rating,Ratings Count,Processor,Screen Size,Manufacturer Color,Color,Ram Size,SSD Capacity,...,Maximum Resolution,Model,OS,Hard Drive Capacity,Country Region Of Manufacture,Storage Type,Features,Condition,Seller Note,Price
700,85264981608,Google,5 out of 5 stars,12.0,Intel Core i5 8th Gen.,13.3 in,NaN,Black,8 GB,128GB,...,1920 x 1080,Google Pixelbook Go,Chrome OS,NaN,NaN,SSD (Solid State Drive),"Built-in Webcam, Ambient Light Sensor, Backlit...",NaN,NaN,$219.99
701,85264981609,Lenovo,NaN,NaN,Intel Core i7 2nd Gen.,15.6 in,NaN,Black,16 GB,512 GB,...,NaN,Lenovo ThinkPad W520,Windows 10 Pro,NaN,NaN,SSD (Solid State Drive),"Bluetooth, Built-in Microphone, Built-in Webca...",Very Good - RefurbishedThe item shows minimal ...,““Tested working. The Unit shows minor cosmeti...,$268.00
702,85264981610,Dell,NaN,NaN,Intel Core i7 9th Gen.,14 in,NaN,NaN,32 GB,1 TB,...,1920 x 1080,LATITUDE 5401,Windows 11 Pro,1 TB,NaN,SSD (Solid State Drive),Wi-Fi,Excellent - Refurbished: The item is in like-n...,NaN,$399.99
703,85264981611,Google,NaN,NaN,Google Tensor G2,10.95 in,NaN,Gray,8 GB,128 GB,...,2560 x 1600,Google Pixel Tablet GA04754-US,Android,NaN,China,UFS (Universal Flash Storage),"Bluetooth, Built-in Microphone, Built-in Webca...","Open boxAn item in excellent, new condition wi...",“Open-box item! Might have light scratches. Mi...,$299.99
704,85264981612,Lenovo,NaN,NaN,Intel Core i7 4th Gen.,"14.1""",NaN,Purple,16GB,256 GB,...,1600 x 900,ThinkPad T460p,Windows 10 Pro,No,United States,SSD (Solid State Drive),NaN,Used: An item that has been used previously. T...,NaN,$187.77
705,85264981613,Dell,NaN,NaN,Intel Core i7-7600U,"14""",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Used: An item that has been used previously. T...,NaN,$99.99
706,85264981614,ASUS,NaN,NaN,Intel Core i3 11th Gen.,14 in,NaN,Black,8 GB,256 GB,...,1920 x 1080,Asus VivoBook Flip 14,Windows 11 Home,256 GB,NaN,SSD (Solid State Drive),"Touchscreen, Bluetooth, Built-in Webcam, Wi-Fi...",NaN,NaN,$289.49
707,85264981615,Panasonic,NaN,NaN,Core i5-7300U,12 in,NaN,Silver,8 GB,256 GB,...,2160 x 1440,Let's note CF-XZ6RF7VS,Windows 10 Pro,8 GB,Japan,SSD (Solid State Drive),LTE,Used: An item that has been used previously. T...,NaN,$178.00
708,85264981616,Panasonic,NaN,NaN,Qualcomm MSM8909,7 in,NaN,NaN,NaN,16 GB,...,NaN,NaN,ANDROID,NaN,NaN,eMMC,"Rugged, Wi-Fi",UsedAn item that has been used previously. The...,“MINT GRADE A - Ex- DEMO BOXED”,$208.73
709,85264981617,Microsoft,NaN,NaN,Intel Core i5 6th Gen.,12.3 in,NaN,Silver,8 GB,256 GB,...,NaN,Microsoft Surface Pro,Windows 10 Pro,256 GB,NaN,SSD (Solid State Drive),NaN,UsedAn item that has been used previously. The...,“Used. Tested/Inspected. Does not come with ke...,$129.99


To me, I think ebay would benefit from finding determinants of product price. So, that is the framework I'm going to use when cleaning the data.

In [3]:
# print out data types. Seems like we have no important numeric values.. ratings count may end up being removed and item number is a key.
ebay_data.dtypes

Item Number                        int64
Brand                             object
Rating                            object
Ratings Count                    float64
Processor                         object
Screen Size                       object
Manufacturer Color                object
Color                             object
Ram Size                          object
SSD Capacity                      object
GPU                               object
Processor Speed                   object
Type                              object
Release Year                      object
Maximum Resolution                object
Model                             object
OS                                object
Hard Drive Capacity               object
Country Region Of Manufacture     object
Storage Type                      object
Features                          object
Condition                         object
Seller Note                       object
Price                             object
dtype: object

In [4]:
# Use the .sum() function get a count of the total number of missing values
missing_value_counts = ebay_data.isna().sum()

# we can convert those numbers into percentages by dividing the counts by the number of records in
# the data (i.e., the length)
missing_value_percentages = missing_value_counts/len(ebay_data) * 100
print(missing_value_percentages)

Item Number                       0.000000
Brand                            37.704439
Rating                           95.604556
Ratings Count                    95.604556
Processor                        40.522780
Screen Size                      39.705023
Manufacturer Color               97.079439
Color                            66.486565
Ram Size                         66.486565
SSD Capacity                     56.454439
GPU                              53.008178
Processor Speed                  58.367407
Type                             49.080023
Release Year                     87.324766
Maximum Resolution               61.404790
Model                            50.949182
OS                               52.701519
Hard Drive Capacity              63.960280
Country Region Of Manufacture    96.948014
Storage Type                     62.602220
Features                         60.995911
Condition                         9.433411
Seller Note                      76.985981
Price      

Rating, Ratings Count, Manufacturer Color and Country Region of Manufacture all have missing value % of above 95. For this reason, I am dropping those columns.
Seller Note seems to just be a description of the product. Not really any quantifiable data can be obtained, plus 77% are missing it... so I'm dropping that variable. Release Year is also missing from 87% of the observations. I'm going to drop that variable as well.

If we do this, than the rest of the columns, at most, are missing at most 67% of observations. I think that this framework is appropriate.

I am also going to drop the Features column because it is verbose and doesn't seem to add much to the understanding of the data. By dropping it we get access to 18 more observations that have all of the values in each column.

In [5]:
# create new sample without unnecessary columns
ebay_data_sample = ebay_data.drop(columns=['Rating','Features','Ratings Count','Manufacturer Color','Country Region Of Manufacture','Seller Note','Release Year'])
ebay_data_sample = ebay_data_sample.dropna()
ebay_data_sample.shape

(466, 17)

This leaves us with 466 sampled rows.

Next, I am going to go through the columns and correct any messy values. This will take a long time, but the work of narrowing the dataset to a sample is already done.

In [6]:
# get list of unique values for Brand column
brand_unique_values = ebay_data_sample['Brand'].unique()
# sort values
brand_unique_values.sort()
brand_unique_values

array(['ASUS', 'Acer', 'CHUWI', 'Chuwi', 'Dell', 'Dell Inc.',
       'Dell Inspiron', 'Fujitsu', 'GPD', 'Gateway', 'HP', 'Jumper', 'LG',
       'Lenovo', 'Lenovo T440', 'MSI', 'Microsoft', 'Panasonic', 'Razer',
       'SGIN', 'Samsung', 'Zzetze'], dtype=object)

In [7]:
# Rename all values "CHUWI" to "Chuwi" as they mean the same thing
ebay_data_sample['Brand'] = ebay_data_sample['Brand'].replace(to_replace='CHUWI',
value='Chuwi')
# Rename all values related to Dell to Dell -- e.g. "Dell Inc.", "Dell Inspiron" as they are synonymous with Dell
ebay_data_sample['Brand'] = ebay_data_sample['Brand'].replace(to_replace=['Dell Inc.', 'Dell Inspiron'],
value='Dell')
# Rename all values related to Lenovo to Lenovo -- e.g. "Lenovo T440"
ebay_data_sample['Brand'] = ebay_data_sample['Brand'].replace(to_replace='Lenovo T440',
value='Lenovo')

In [8]:
# get list of unique values for Color column
color_unique_values = ebay_data_sample['Color'].unique()
# sort values
color_unique_values.sort()
color_unique_values

array(['Abyss Blue', 'Arctic Grey', 'Black', 'Black/Red', 'Blue',
       'Bronze', 'Brown', 'Carbon Black', 'Fog Blue Aluminum', 'Gray',
       'Gray and Black', 'Green', 'Grey', 'Mica Silver', 'Mineral Silver',
       'Multi', 'Multicolor', 'Natural Silver', 'Pure Silver', 'Purple',
       'Quiet Blue', 'Red', 'STEEL GRAY', 'Silver', 'Silver / Black',
       'Silver and Black', 'Silver/Black', 'Standard', 'Warm Gold',
       'White'], dtype=object)

We have lots of similar-looking colors (e.g. Natural Silver and Silver), and duplicates (Silver / Black and Silver/Black). We will change these values for larger subgroups.

In [9]:
# going to rearrange colors so that we take the common color denominator for specific colors, this will make the data more uniform
ebay_data_sample['Color'] = ebay_data_sample['Color'].replace({
    'Arctic Grey': 'Grey',
    'Grey': 'Gray',
    'Natural Silver': 'Silver',
    'Pure Silver': 'Silver',
    'STEEL GRAY': 'Gray',
    'Mica Silver': 'Silver',
    'Carbon Black': 'Black',
    'Mineral Silver': 'Silver',
    'Abyss Blue': 'Blue',
    'Quiet Blue': 'Blue',
    'Fog Blue Aluminum': 'Blue',
    'Multi': 'Multicolor',
    'Silver / Black': 'Silver/Black',
    'Silver and Black': 'Silver/Black',
    'Gray and Black': 'Gray/Black',
})

In [10]:
## rename column to include units
ebay_data_sample.rename(columns={'Processor Speed':'Processor Speed (GHz)'}, inplace=True)

# get list of unique values for Processor Speed column
processor_speed_unique_values = ebay_data_sample['Processor Speed (GHz)'].unique()
# sort values
processor_speed_unique_values.sort()
processor_speed_unique_values

array(['1.00 GHz', '1.00 GHz.83ghz', '1.1 GHz', '1.10 GHz', '1.10GHz',
       '1.1GHz-2.4Ghz', '1.20 GHz', '1.40 GHz', '1.50 GHz', '1.6 GHz',
       '1.60 GHz', '1.60GHz', '1.7 GHz', '1.70 GHz', '1.73 GHz',
       '1.80 GHz', '1.80 Ghz', '1.83 GHz', '1.9 GHz', '1.9 Ghz',
       '1.90 GHz', '1.90GHz', '2.00 GHz', '2.00GHz', '2.10 GHz',
       '2.16 GHz', '2.20 GHz', '2.3', '2.30 GHz', '2.30 GHz / 4.60 GHz',
       '2.30Ghz', '2.3GHz', '2.4', '2.40 GHz', '2.48 GHz', '2.4ghz',
       '2.50 GHz', '2.5GHz', '2.60 GHz', '2.67 GHz', '2.6GHz', '2.70 GHz',
       '2.80 - 4.70 GHz', '2.80 GHz', '2.8GHz-3.8GHz', '2.90 GHz',
       '2.90GHz', '3.00 GHz', '3.10 GHz',
       '3.10 GHz (2.50 GHz Base Frequnecy)', '3.30 GHz', '3.40 GHz',
       '3.50 GHz', '3.60 GHz', '3.60 GHz (1.70 GHz Base Frequency)',
       '3.80 GHz', '3.90 GHz', '3.90 GHz (2.80 GHz Base Frequency)',
       '4.10 GHz', '4.20 GHz', '4.30 GHz', '4.6 GHz', '4.60 GHz',
       '4.70 GHz', '4.7GHz', '4.90 GHz (1.80 GHz Base Frequency)

First, I want to convert Processor Speed values with units into simple numeric values -- because all of these observations are the same unit, trimming the unit and putting it in the column title will make it easier to explore in a numeric context.

For the processor speed, I am going to take the maximum value in any range. For example, "3.10 GHz (2.50 GHz Base Frequency)" would be equivalent to "3.10" because I am assuming that the max processor speed is listed for all products that do not include a range. Same goes for entries like "Up to 3.3GHz" ("3.30") and "1.1Ghz-2.4Ghz" (which would be "2.40"). I am also going to make sure that there are two decimals for each measurement for consistency of data.

In [11]:
# remove units terms and remove unnecessary strings en masse
ebay_data_sample['Processor Speed (GHz)'] = ebay_data_sample['Processor Speed (GHz)'].replace(to_replace=[
    r'\s*GHz',r'\s*Ghz',r'\s*ghz',r'\s*GHZ',r'\s*GHz.83ghz'], value='', regex=True)
ebay_data_sample['Processor Speed (GHz)'] = ebay_data_sample['Processor Speed (GHz)'].replace(to_replace=[
    r'\s*up to ',r'\s*UP to ',r'\s*Frequency',r'\s*Frequnecy',r'\s*Up to '], value='', regex=True)

In [12]:
# Convert strings to float-convertible strings, and make adjustments based on previously stated assumptions
ebay_data_sample['Processor Speed (GHz)'] = ebay_data_sample['Processor Speed (GHz)'].replace({
    '1.00.83': '1.00',
    '1.1-2.4': '2.40',
    '2.30 / 4.60': '4.60',
    '2.8-3.8': '3.80',
    '2.80 - 4.70': '4.70',
    '3.10 (2.50 Base)': '3.10',
    '3.60 (1.70 Base)' : '3.60',
    '3.90 (2.80 Base)': '3.90',
    '4.90 (1.80 Base)': '4.90',
    'Max Turbo @ 3.90': '3.90',
    'Max3.80': '3.80',
    'I5-8350u @1.7': '1.7'
})

# Convert every value in the 'Processor Speed (GHz)' column to a float with two decimal points
ebay_data_sample['Processor Speed (GHz)'] = ebay_data_sample['Processor Speed (GHz)'].astype(float)

In [13]:
# examine all of the unique screen size values. we are hoping to convert them to floats for easier data visualization
screen_size_unique_values = ebay_data_sample['Screen Size'].unique()
screen_size_unique_values

array(['14 in', '14.1 in', '11 in', '11.6 in', '12.5 in', '15.6 in',
       '13.5 in', '14.1in.', '15.6 inches', '14.1"', '17.3 in', '16.1 in',
       '10.8 in', '13.3 in', '13 in', '16.0 in', '14" HD Matte Display',
       '10.5 in', '14 inch', '17 in', '15.4 in', '7 in', '13.3"', '15 in',
       '11.6"', '15.6"', '14 In', '16 in', '15.6 In', '12 in', '14.5 in',
       '12.5" FHD (1920x1080)', '11.5 in', '14in.', '12.3 in', '14',
       '13.1 in', '10.1 in', '12.1 in', '15.6 in FHD', '15.6', '12.5in.'],
      dtype=object)

We can see a lot of "in" measurements and variations of it denoting inches (which is the screen size.) I am going to convert it to a float by first removing all of the extra non-numeric characters. I will also correct any typos...

In [14]:
# replace all the added strings in the screen size column, and convert all of the values to floats
ebay_data_sample['Screen Size'] = ebay_data_sample['Screen Size'].replace(to_replace=[
    r'\s*inches',
    r'\s*in.',
    r'\s*In',
    r'\s*in',
    r'\s*HD Matte Display',
    r'\s*FHD',
    r'\s*h',
    r'\s*"',
    r'\(1920x1080\)'], value='', regex=True) 
# float conversion:                                                                                    
ebay_data_sample['Screen Size'] = ebay_data_sample['Screen Size'].apply(lambda x: float(f"{float(x):.2f}"))

In [15]:
# now to rename the column so that the viewer can see that all of the observations are measured in inches
ebay_data_sample.rename(columns={'Screen Size':'Screen Size (in)'}, inplace=True)
ebay_data_sample.iloc[461]

Item Number                                                    85264987126
Brand                                                                   HP
Processor                                          Intel Core i7 13th Gen.
Screen Size (in)                                                      17.3
Color                                                               Silver
Ram Size                                                             64 GB
SSD Capacity                                                          4 tb
GPU                                                 Intel Iris Xe Graphics
Processor Speed (GHz)                                                  1.7
Type                                                       Notebook/Laptop
Maximum Resolution                                             1920 x 1080
Model                                                       HP 17-cn3097nr
OS                                                          Windows 11 Pro
Hard Drive Capacity      

Notice how we have a range for price in this index location? Since I've been picking the maximum of the range in any ranges we've seen in previous variables, I am going to pick the maximum price. It's likely that this is because the laptops are customizable once you select them online, so by selecting the most advanced qualities for those with ranges, like storage or RAM, I assume we'll end up with the highest price anyway.

In [16]:
# find all prices that have a range and select the maximum of the range to improve the observation
ebay_data_sample['Price'] = ebay_data_sample['Price'].apply(lambda x: x.split(' to ')[1] if 'to' in x else x)

# time to take price and turn it into a float variable. First, find the $ and , in each price key and remove it
for i in range(len(ebay_data_sample['Price'])):
    ebay_data_sample['Price'].iloc[i] = ebay_data_sample['Price'].iloc[i].replace('$', '')
    ebay_data_sample['Price'].iloc[i] = ebay_data_sample['Price'].iloc[i].replace(',', '')

# then, convert to float and rename column to include units of measurement
ebay_data_sample['Price'] = ebay_data_sample['Price'].astype(float)
ebay_data_sample.rename(columns={'Price':'Price ($)'}, inplace=True)

C:\Users\aaron\AppData\Local\Temp\ipykernel_35080\3199488211.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  ebay_data_sample['Price'].iloc[i] = ebay_data_sample['Price'].iloc[i].replace('$', '')
C:\Users\aaron\AppData\Local\Temp\ipykern

In [17]:
# print out all descriptions -- we are going to remove the lengthy words and narrow it down to a one or two-word description
print(ebay_data_sample['Condition'].unique())

["New: A brand-new, unused, unopened, undamaged item in its original packaging (where packaging is ...  Read moreabout the conditionNew: A brand-new, unused, unopened, undamaged item in its original packaging (where packaging is applicable). Packaging should be the same as what is found in a retail store, unless the item is handmade or was packaged by the manufacturer in non-retail packaging, such as an unprinted box or plastic bag. See the seller's listing for full details. See all condition definitionsopens in a new window or tab "
 'UsedAn item that has been used previously. The item may have some signs of cosmetic wear, but is fully operational and functions as intended. This item may be a floor model or store return that has been used. See the seller’s listing for full details and description of any imperfections. See all condition definitionsopens in a new window or tab'
 "Very Good - RefurbishedThe item shows minimal wear and is backed by a one year warranty. It is fully functio

There are 8 types of conditions here: New, Used, Very Good, Open box, Excellent, For parts or not working, Good, Certified. It is best to remove all the extra description for clarity and readability. There are some conditions with variations based on the presence of a colon.

In [18]:
# Go through and split the data at the first colon or space
ebay_data_sample['Condition'] = ebay_data_sample['Condition'].apply(lambda x: x.split(':')[0] if ':' in x else x.split(' ')[0])
print(ebay_data_sample['Condition'].unique())

['New' 'UsedAn' 'Very' 'Used' 'Very Good - Refurbished' 'Open' 'Excellent'
 'For' 'Excellent - Refurbished' 'Certified' 'Open box' 'Good'
 'For parts or not working' 'Good - Refurbished']


In [19]:
# correct some typos to match the original 8 characteristic types
ebay_data_sample['Condition'] = ebay_data_sample['Condition'].replace({
    'UsedAn': 'Used',
    'Very': 'Very Good',
    'Excellent - Refurbished': 'Excellent',
    'Good - Refurbished': 'Good',
    'Very Good - Refurbished': 'Very Good',
    'Open': 'Open box',
    'For' : 'For parts or not working'
})

In [20]:
# print out unique values in the 'Storage Type' column
storage_type_unique_values = ebay_data_sample['Storage Type'].unique()
storage_type_unique_values

array(['SSD (Solid State Drive)', 'HDD + SSD', 'eMMC', 'SSD',
       'HDD (Hard Disk Drive)', 'NVMe', 'M.2 SOLID STATE', 'SSD + eMMC',
       'eMMC Solid State SSD', 'Touchscreen', '256 G', '.',
       'Solid State Drive (SSD)', 'SSD + HDD', 'NVMe m.2'], dtype=object)

In [21]:
# change SSD (Solid State Drive) to SSD and HDD (Hard Disk Drive) to HDD.. as well as combine the HDD + SSD and SSD + HDD values as they mean the same thing
# it also looks like eMMC Solid State SSD and SSD + eMMC equal the same thing.. 
# after doing research, M.2 is a type of SSD and NVMe is a type of M.2, so I will be making sure they fall under their common denominators.
# "Most NVMe are M.2, not all M.2 use NVMe."

ebay_data_sample['Storage Type'] = ebay_data_sample['Storage Type'].replace({
    'SSD (Solid State Drive)': 'SSD',
    'HDD (Hard Disk Drive)': 'HDD',
    'HDD + SSD': 'SSD + HDD',
    'Solid State Drive (SSD)': 'SSD',
    'eMMC Solid State SSD': 'SSD + eMMC',
    'NVMe m.2': 'NVMe',
    'M.2 SOLID STATE' : 'M.2'
})

storage_type_unique_values

array(['SSD (Solid State Drive)', 'HDD + SSD', 'eMMC', 'SSD',
       'HDD (Hard Disk Drive)', 'NVMe', 'M.2 SOLID STATE', 'SSD + eMMC',
       'eMMC Solid State SSD', 'Touchscreen', '256 G', '.',
       'Solid State Drive (SSD)', 'SSD + HDD', 'NVMe m.2'], dtype=object)

Looks like we may have to investigate and/or drop rows containing unrelated values for this category -- "touchscreen" is not a storage type, and neither is "256 G", which is most likely an error. We also have a single "." which should be removed too.

In [22]:
ebay_data_sample = ebay_data_sample[ebay_data_sample['Storage Type'] != '.']
ebay_data_sample = ebay_data_sample[ebay_data_sample['Storage Type'] != '256 G']
ebay_data_sample = ebay_data_sample[ebay_data_sample['Storage Type'] != 'Touchscreen']
print("We now have "+ str(len(ebay_data_sample)) + " observations.")

We now have 461 observations.


In [23]:
# time to tackle the 'Hard Drive Capacity' column by adding a unit to the title and retrieving unique values
ebay_data_sample.rename(columns={'Hard Drive Capacity':'Hard Drive Capacity (GB)'}, inplace=True)
hd_capacity_unique_values = ebay_data_sample['Hard Drive Capacity (GB)'].unique()
hd_capacity_unique_values.sort()
hd_capacity_unique_values

array(['0', '0GB', '1 GB', '1 TB', '12 GB', '120 GB', '128 GB', '128GB',
       '128GB,  256GB,  500GB,  1TB,  2TB', '16 GB', '160 GB', '1TB',
       '1tb', '1tb nvme', '2 TB', '240 GB', '250 GB', '256 GB',
       '256 GB SSD', '256GB', '2TB', '32 GB', '320 GB', '32GB', '4 GB',
       '4 TB', '500 GB', '512 GB', '512 GB SSD', '512GB', '512gb',
       '64 GB', '64GB', '64gb', '8 GB', "Don't Apply", 'No', 'No HDD',
       'Not Applicable', 'SSD Only', 'SSD only', 'Up to 1 TB',
       'Up to 2 TB', 'Up to 2TB'], dtype=object)

In [24]:
# proceed by removing non-numeric substrings
ebay_data_sample['Hard Drive Capacity (GB)'] = ebay_data_sample['Hard Drive Capacity (GB)'].replace(to_replace=[
    r'\s*Up to ',r'\s*GB',r'\s*SSD',r'\s*gb',r'\s*nvme'], value='', regex=True)
ebay_data_sample['Hard Drive Capacity (GB)'] = ebay_data_sample['Hard Drive Capacity (GB)'].replace({
    # the following two conversions are for originally named 'SSD Only" entries -- since they are SSD only, we assume they have no hard drive (HDD) capacity.
    ' Only': '0',
    ' only': '0',
    '1 TB': '1000',
    '1TB': '1000',
    '1tb': '1000',
    '2 TB': '2000',
    '2TB': '2000',
    # taking the max value from these options to be consistent with our previous methods
    '128,  256,  500,  1TB,  2TB' : '2000',
    '4 TB': '4000',
    # converting these entries to zero
    "Don't Apply": '0',
    'No': '0',
    'No HDD': '0',
    'Not Applicable': '0'
})
ebay_data_sample['Hard Drive Capacity (GB)'] = ebay_data_sample['Hard Drive Capacity (GB)'].astype(int)

In [25]:
# time to tackle the 'SSD Capacity' column by adding a unit to the title and retrieving unique values
ebay_data_sample.rename(columns={'SSD Capacity':'SSD Capacity (GB)'}, inplace=True)
ssd_capacity_unique_values = ebay_data_sample['SSD Capacity (GB)'].unique()
ssd_capacity_unique_values.sort()
ssd_capacity_unique_values

array(['0', '1 TB', '1 TB(not in)',
       '1 Used,  1 support Max 1TB M.2 Extend', '120 GB', '120GB',
       '128 GB', '128GB', '128GB,  256GB,  500GB,  1TB,  2TB', '16 GB',
       '16GB', '1tb', '2 TB', '240GB', '256 GB', '256 GBSSD', '256GB',
       '256SSD', '2TB', '32 GB', '32GB', '4 tb', '500 GB', '512',
       '512 GB', '512 GB SSD', '512GB', '64 GB', '64gb', '931.51GB',
       'NONE', 'Not Applicable', 'Not Include', 'Up to 1 TB',
       'Up to 2 TB', 'Up to 2TB', 'na'], dtype=object)

I am going to remove all "GB" terms, first off. Then I will convert the none/non applicable values and convert them to 0, as that is mathematically true. For the '1TB (not in)' term, I am assuming that the "not in" adjustment signifies that it is not included with the order. This is supported by the fact that there is a "Not Include" term, which means that is a possibility for it to not be included.
As I've mentioned, any "Up to" subscripts will be removed under the assumption that we are looking at the maximum value for each field where there are multiple values.

In [26]:
# remove substrings
ebay_data_sample['SSD Capacity (GB)'] = ebay_data_sample['SSD Capacity (GB)'].replace(to_replace=[
    r'\s*Up to ',r'\s*GB',r'\s*SSD',r'\s*gb'], value='', regex=True)

In [27]:
# replace terms
ebay_data_sample['SSD Capacity (GB)'] = ebay_data_sample['SSD Capacity (GB)'].replace({
    '1 TB': '1000',
    '1 TB(not in)': '0',
    # max of 1TB --> 1000
    '1 Used,  1 support Max 1TB M.2 Extend': '1000',
    # taking the max value from these options to be consistent with our previous methods
    '128,  256,  500,  1TB,  2TB': '2000',
    '1tb': '1000',
    '2 TB': '2000',
    '2TB': '2000',
    '4 tb': '4000',
    "NONE": '0',
    'Not Include': '0',
    'na': '0',
    'Not Applicable': '0'})
ebay_data_sample['SSD Capacity (GB)'] = ebay_data_sample['SSD Capacity (GB)'].astype(float)

In [28]:
# peruse the OS types to correct any typos and combine like terms
OS_unique_values = ebay_data_sample['OS'].unique()
OS_unique_values.sort()
OS_unique_values

array(['Chrome OS', 'Linux', 'Microsoft Windows 10 Professional',
       'Microsoft Windows 11', 'Not Included', 'Pop OS', 'Win 10 Pro 64',
       'Windows 10', 'Windows 10 Home', 'Windows 10 Home 64',
       'Windows 10 Pro', 'Windows 10 Professional', 'Windows 11',
       'Windows 11 Home', 'Windows 11 Home S', 'Windows 11 Pro',
       'Windows 11 S Mode', 'Windows 11 S mode', 'Windows 7',
       'Windows 7 Professional', 'Windows XP', 'windows 11',
       '\u200eWindows 11 Home'], dtype=object)

I am now going to clean up these terms.

In [29]:
ebay_data_sample['OS'] = ebay_data_sample['OS'].replace({
    'Microsoft Windows 10 Professional': 'Windows 10 Pro',
    'Windows 10 Professional': 'Windows 10 Pro',
    # it appears after some research online that S mode is a part of the Home feature. changing all S modes to Home S for consistency
    'Windows 11 S mode': 'Windows 11 Home S',
    'Windows 11 S Mode': 'Windows 11 Home S',
    'windows 11': 'Windows 11',
    'Microsoft Windows 11': 'Windows 11',
    "\u200eWindows 11 Home": 'Windows 11 Home',
    'Windows 7 Professional': 'Windows 7 Pro',
    "Win 10 Pro 64": 'Windows 10 Pro 64',
    # after more research, I am going to combine the pro 64 with pro and home 64 with home, as a vast majority of new models
    # use the 64-bit operating system even if not explicitly titled. home and pro are general designations.
    "Windows 10 Pro 64": 'Windows 10 Pro',
    "Windows 10 Home 64": 'Windows 10 Home',
    # lastly, it seems that Home S is a variation of Home, so I will change the Home S to Home in any key.
    "Windows 11 Home S": 'Windows 11 Home'
    })

In [30]:
# time to tackle the 'SSD Capacity' column by adding a unit to the title and retrieving unique values
ebay_data_sample.rename(columns={'Ram Size':'Ram Size (GB)'}, inplace=True)
ram_unique_values = ebay_data_sample['Ram Size (GB)'].unique()
ram_unique_values.sort()
ram_unique_values

array(['12 GB', '12GB', '12gb', '16 GB', '16 GB DDR4', '16GB', '2 GB',
       '20 GB', '20gb', '24 GB', '24GB', '32 GB', '4 GB', '4 MB', '4GB',
       '6 GB', '64 GB', '64 MB', '8 GB',
       '8 GB DDR4 Installed---Can upgrade o 32GB DDR4', '8GB DDR4',
       '8GB,  16GB,  32GB,  64GB', 'Up to 32 GB', 'Up to 32GB',
       'Up to 36 GB', 'Up to 40GB', 'Up to 64GB'], dtype=object)

In [31]:
# remove substrings
ebay_data_sample['Ram Size (GB)'] = ebay_data_sample['Ram Size (GB)'].replace(to_replace=[
    r'\s*Up to ',r'\s*GB',r'\s*DDR4',r'\s*gb'], value='', regex=True)
ebay_data_sample['Ram Size (GB)'] = ebay_data_sample['Ram Size (GB)'].replace({
    # update value to maximum available value which is 32
    '8 Installed---Can upgrade o 32': '32',
    # update value to maximum available value which is 64
    '8,  16,  32,  64': '64',
    # convert MB measurements to GB -- 1 MB = 0.001 GB
    '4 MB': '0.004',
    '64 MB': '0.064',
    })
ebay_data_sample['Ram Size (GB)'] = ebay_data_sample['Ram Size (GB)'].astype(float)

In [32]:
# retrieve resolution unique values
res_unique_values = ebay_data_sample['Maximum Resolution'].unique()
res_unique_values.sort()
res_unique_values

array(['1024 x 480', '1024 x 728', '1024 x 768', '1028 x 800',
       '1280 x 800', '1366 x 768', '1366 x 786', '1366x768', '1600 x 900',
       '1680 x 1050', '1900x600', '1920 x 1080', '1920 x 1080p',
       '1920 x 1200', '1920 x 1280', '1920x1080', '2160 x 1440',
       '2256 x 1504', '2560 x 1440', '2560 x 1600', '2560x1600',
       '2736 x 1824', '2880 x 1800', '2880 x 1920', '3000 x 2000',
       '3840 x 2160', '3840 x 2400', 'Full HD (1920 x 1080)',
       'Full HD (1920x1080p)', 'HD (1366 x 768)'], dtype=object)

We have a lot of unique values here, even if we clean up all that we can.

In [33]:
ebay_data_sample['Maximum Resolution'] = ebay_data_sample['Maximum Resolution'].replace({
    'Full HD (1920x1080p)': '1920 x 1080',
    'Full HD (1920 x 1080)': '1920 x 1080',
    '1920x1080': '1920 x 1080',
    '1920 x 1080p': '1920 x 1080',
    '1900x600': '1900 x 600',
    # it appears after some online research that the '786' term should be '768' and that '786' was a typo
    '1366 x 786': '1366 x 768',
    '1366x768': '1366 x 768',
    'HD (1366 x 768)': '1366 x 768',
    '2560x1600': '2560 x 1600'
    })

Because we still have so many values, and they don't format numerically, I would like to add another key to our data. I want to create a "resolution size" dummy variable with 3 descriptions: "small", "medium", and "large". To do this, I am going to calculate the total number of pixels for each unique resolution measurement, and then categorize them by taking the 33rd and 66th percentiles of numeric resolution size and dividing my characterizations of resolution size among those lines. This will give us a new variable that is easier to conduct analysis with instead of the multitude of resolution sizes that we currently have.

In [34]:
# create new tuple that has a product of the pixel resolution sizes
pixel_areas = [(int(res.split(' x ')[0]) * int(res.split(' x ')[1]), res) for res in ebay_data_sample['Maximum Resolution']]
pixel_areas

[(3110400, '2160 x 1440'),
 (2073600, '1920 x 1080'),
 (2073600, '1920 x 1080'),
 (3110400, '2160 x 1440'),
 (1049088, '1366 x 768'),
 (1049088, '1366 x 768'),
 (2073600, '1920 x 1080'),
 (2073600, '1920 x 1080'),
 (2073600, '1920 x 1080'),
 (2073600, '1920 x 1080'),
 (2073600, '1920 x 1080'),
 (3393024, '2256 x 1504'),
 (2073600, '1920 x 1080'),
 (1049088, '1366 x 768'),
 (1049088, '1366 x 768'),
 (2073600, '1920 x 1080'),
 (2073600, '1920 x 1080'),
 (2073600, '1920 x 1080'),
 (2073600, '1920 x 1080'),
 (2073600, '1920 x 1080'),
 (2073600, '1920 x 1080'),
 (2073600, '1920 x 1080'),
 (2304000, '1920 x 1200'),
 (2304000, '1920 x 1200'),
 (2073600, '1920 x 1080'),
 (2073600, '1920 x 1080'),
 (3686400, '2560 x 1440'),
 (3686400, '2560 x 1440'),
 (1049088, '1366 x 768'),
 (1049088, '1366 x 768'),
 (2073600, '1920 x 1080'),
 (2073600, '1920 x 1080'),
 (2073600, '1920 x 1080'),
 (2073600, '1920 x 1080'),
 (2073600, '1920 x 1080'),
 (2073600, '1920 x 1080'),
 (2304000, '1920 x 1200'),
 (23040

In [35]:
# turn this into a data frame
df = pd.DataFrame(pixel_areas, columns=['Area','Maximum Resolution'])

# Calculate the 33rd and 66th percentiles
percentile_33 = np.percentile(df['Area'], 33)
percentile_66 = np.percentile(df['Area'], 66)

# Define a function to categorize each area
def categorize_area(area):
    if area <= percentile_33:
        return 'Small'
    elif area <= percentile_66:
        return 'Medium'
    else:
        return 'Large'

# Apply the function to the DataFrame to create a new column
df['Resolution Size'] = df['Area'].apply(categorize_area)

# drop the redundant "area" columns
df = df.drop(columns=['Area'])

# drop any duplicates in our data frame to make for easier and more simple merging
df = df.drop_duplicates(subset=['Maximum Resolution'])

# Display the DataFrame with the new column
print(df)

    Maximum Resolution Resolution Size
0          2160 x 1440           Large
1          1920 x 1080          Medium
4           1366 x 768           Small
11         2256 x 1504           Large
22         1920 x 1200           Large
26         2560 x 1440           Large
51          1600 x 900           Small
67          1024 x 768           Small
93         2880 x 1920           Large
94         3000 x 2000           Large
104        2560 x 1600           Large
108        3840 x 2160           Large
144        1680 x 1050          Medium
185        3840 x 2400           Large
238         1028 x 800           Small
291        2736 x 1824           Large
315         1900 x 600           Small
331         1024 x 728           Small
340        1920 x 1280           Large
385         1280 x 800           Small
395         1024 x 480           Small
411        2880 x 1800           Large


In [36]:
# merge the data frame with our sample to add that new column
ebay_data_sample = pd.merge(ebay_data_sample, df, on='Maximum Resolution', how='left')
ebay_data_sample

,Item Number,Brand,Processor,Screen Size (in),Color,Ram Size (GB),SSD Capacity (GB),GPU,Processor Speed (GHz),Type,Maximum Resolution,Model,OS,Hard Drive Capacity (GB),Storage Type,Condition,Price ($),Resolution Size
0,85264981001,Chuwi,Quad Core,14.0,Gray,8.0,1000.0,Intel Iris Plus Graphics 655,3.8,Notebook/Laptop,2160 x 1440,CoreBook X,Windows 11 Home,512,SSD,New,303.68,Large
1,85264981003,Dell,Intel Core i5-6300U,14.0,Black,16.0,500.0,Intel HD Graphics,2.4,Notebook/Laptop,1920 x 1080,Dell Latitude E5470,Windows 10 Pro,500,SSD,Used,175.00,Medium
2,85264981014,Chuwi,Intel Celeron Processor N4100,14.1,Gray,8.0,256.0,"Intel UHD Graphics 600, 650MHz",2.8,Notebook/Laptop,1920 x 1080,CHUWI Lapbook Pro,Windows 11 Home,256,SSD,New,151.99,Medium
3,85264981015,Chuwi,Intel Core i5-8259U,14.0,Gray,8.0,512.0,Intel Iris Plus Graphics 655,3.8,Notebook/Laptop,2160 x 1440,CHUWI CoreBook X,Windows 10,512,SSD + HDD,New,243.99,Large
4,85264981018,Dell,Intel Celeron,11.0,Black,4.0,128.0,Intel UHD Graphics 600,1.1,Notebook/Laptop,1366 x 768,Dell Latitude 3189,Windows 10 Pro,128,SSD,Used,129.99,Small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,85264987126,HP,Intel Core i7 13th Gen.,17.3,Silver,64.0,4000.0,Intel Iris Xe Graphics,1.7,Notebook/Laptop,1920 x 1080,HP 17-cn3097nr,Windows 11 Pro,1000,SSD + HDD,New,1280.00,Medium
457,85264987127,ASUS,AMD Ryzen 7 5800HS,16.0,Blue,16.0,1000.0,AMD Radeon Graphics,4.4,Notebook/Laptop,1920 x 1200,ASUS VivoBook,Windows 11 Home,1000,SSD,New,609.00,Large
458,85264987136,Lenovo,AMD Ryzen 5,14.0,Black,16.0,256.0,AMD Radeon Graphics,2.3,Notebook/Laptop,1920 x 1080,Lenovo ThinkPad T14,Windows 10 Pro,256,SSD,New,499.99,Medium
459,85264987139,Lenovo,Intel Core i5-1135G7,15.6,Grey,20.0,1000.0,Intel Iris Xe Graphics,4.2,Notebook/Laptop,1920 x 1080,Lenovo Ideapad 3i,Windows 11 Home,1000,SSD,New,599.00,Medium


I would still like to move our new column so that it is right next to our "maximum resolution" column. I will do that now.

In [37]:
# Get the list of current columns
columns = list(ebay_data_sample.columns)

# Remove 'Resolution Size' from its current position
columns.remove('Resolution Size')

# Insert 'Resolution Size' at the 11th position (index 10)
columns.insert(10, 'Resolution Size')

# Reorder the DataFrame using the new column order
ebay_data_sample = ebay_data_sample[columns]

In [38]:
# retrieve unique values in the processor column
processor_unique_values = ebay_data_sample['Processor'].unique()
processor_unique_values.sort()
processor_unique_values

array(['11 Gen intel Core i3', '11th Generation Intel® Core™ i7-1165G7',
       '12th Gen Intel Core i5-12450H', 'AMD', 'AMD 3015Ce',
       'AMD A4-9120C', 'AMD A6-7310 APU', 'AMD E-Series', 'AMD Ryzen 3',
       'AMD Ryzen 5', 'AMD Ryzen 5-3550H', 'AMD Ryzen 7',
       'AMD Ryzen 7 5800HS', 'AMD Ryzen 7 7840HS',
       'AMD Ryzen 7 Pro 3700U', 'AMD Ryzen Pro 3', 'INTEL Core i7-7Y75',
       'Intel Celeron', 'Intel Celeron N', 'Intel Celeron N2840',
       'Intel Celeron N2940 1.83GHz Quad', 'Intel Celeron N3050',
       'Intel Celeron N3050 1.6GHz Dual-Core', 'Intel Celeron N3060',
       'Intel Celeron N3350', 'Intel Celeron N4020',
       'Intel Celeron N4100', 'Intel Celeron N4500',
       'Intel Celeron N5100', 'Intel Celeron Processor N4100',
       'Intel Core 2 Duo', 'Intel Core Duo', 'Intel Core I5-8300H',
       'Intel Core I7', 'Intel Core M3', 'Intel Core i3',
       'Intel Core i3 10th Gen.', 'Intel Core i3 11th Gen.',
       'Intel Core i3 12th Gen.', 'Intel Core i3 4th 

Looks like we have a ton of different processors. Because it would be tedious to go through make these values uniform, and we'd still be left with tons of different values, we have to consider what the benefits of this column really is. I am not going to drop this column, as it doesn't take away from our data, but the processor speed is far more informative and in numeric form already.

I will still try to make the data as uniform as possible by changing terms to common denominator terms.

In [39]:
# correct some of the strangely or messily worded processor model descriptions
ebay_data_sample['Processor'] = ebay_data_sample['Processor'].replace({
    'Intel i7 8th Gen': 'Intel Core i7 8th Gen.', 'Intel Core i7 8th Gen': 'Intel Core i7 8th Gen.',
    'Intel Core i5 4th Gen. 2.70GHz': 'Intel Core i5 4th Gen.', 'Intel Celeron N3050 1.6GHz Dual-Core': 'Intel Celeron N3050',
    '11 Gen intel Core i3': 'Intel Core i3 11th Gen.',  'intel core i5 8th gen': 'Intel Core i5 8th Gen.',
    # make any i3 model into i3, the i3 component is more informative than the specific model itself
    'Intel Core i3-1115G4': 'Intel Core i3', 'i7-3630qm@2.4ghz': 'Intel Core i7',
    'Intel Celeron Processor N4100': 'Intel Celeron N4100', 'Intel i5': 'Intel Core i5',
    # make any i5 model into i5, the i5 component is more informative than the specific model itself
    'Intel Core I5-8300H': 'Intel Core i5', 'Intel Core i5-4300M CPU @ 1.90 GHz': 'Intel Core i5', 
    'Intel Core i5-4300M CPU @ 2.30 GHz': 'Intel Core i5','11th Generation Intel® Core™ i7-1165G7': 'Intel Core i7 11th Gen.',
    'Intel Core i7-4700 @ 2.4 GHz': 'Intel Core i7','Intel core i7 4th gen': 'Intel Core i7 4th Gen.',
    'core i5 8th gen': 'Intel Core i5 8th Gen.','Intel Core i5-4300Y': 'Intel Core i5',
    'Intel Core i5-1135G7': 'Intel Core i5','Intel Core i5-5300U CPU @ 2.30 GHz': 'Intel Core i5',
    'Intel Core i5-8259U': 'Intel Core i5','Intel Core i5-6200U': 'Intel Core i5',
    'intel Core i5-5300U': 'Intel Core i5','AMD Ryzen 7 7840HS': 'AMD Ryzen 7', 'AMD Ryzen 7 5800HS': 'AMD Ryzen 7','Ryzen 5 5625U': 'AMD Ryzen 5',
    'AMD Ryzen 5-3550H': 'AMD Ryzen 5','Intel Core i3-1215U': 'Intel Core i3', 'Intel Core I7': 'Intel Core i7','Intel i3-5020u': 'Intel Core i3',
    'Intel Core i5-6300U': 'Intel Core i5', '12th Gen Intel Core i5-12450H': 'Intel Core i5 12th Gen.',
    # make any i7 model into i7, the i7 component is more informative than the specific model itself
    'INTEL Core i7-7Y75': 'Intel Core i7', 'i5-7300HQ': 'Intel Core i5', 'i7-1165G7': 'Intel Core i7',
    'i5-11500H': 'Intel Core i5', 'i5-4200m': 'Intel Core i5', 'Intel Core i7-8665U': 'Intel Core i7',
    'Intel Quad i7-6920HQ @2.90GHz': 'Intel Core i7', 'Intel i5-7200U @2.50 GHz,  2701 Mhz,  2Core(s)': 'Intel Core i5',
    'i7-6600U': 'Intel Core i7', 'i7-7820HQ': 'Intel Core i7', 'i7-8550U': 'Intel Core i7', 'Intel i5-2520m': 'Intel Core i5',
    'i5-7200U': 'Intel Core i5', 'Intel i7 Quad-Core': 'Intel Core i7', 'Intel Celeron N2940 1.83GHz Quad': 'Intel Celeron N2940',
    'AMD Ryzen 7 Pro 3700U': 'AMD Ryzen 7 Pro', 'Intel Core m3 7th Gen.':'Intel Core M3','Quad-Core Intel N100':'Intel N100',
    # it looks like intel core 2 duo and intel core duo mean the same thing, after some online research
    'intel N100':'Intel N100', 'Intel Core 2 Duo':'Intel Core Duo'
    })

In fact, I think I am going to convert any generation of i3 or i5 or i7 into the standard description. While there may be small improvements from generation to generation, ultimately the big difference is between it being an i3, i5, or i7 processor. Same for Pentium and Celeron -- i am going to combine the subterms (e.g. Pentium Gold, Pentium N4200 into Pentium.)

In [40]:
processor_unique_values = ebay_data_sample['Processor'].unique()
processor_unique_values.sort()
processor_unique_values

array(['AMD', 'AMD 3015Ce', 'AMD A4-9120C', 'AMD A6-7310 APU',
       'AMD E-Series', 'AMD Ryzen 3', 'AMD Ryzen 5', 'AMD Ryzen 7',
       'AMD Ryzen 7 Pro', 'AMD Ryzen Pro 3', 'Intel Celeron',
       'Intel Celeron N', 'Intel Celeron N2840', 'Intel Celeron N2940',
       'Intel Celeron N3050', 'Intel Celeron N3060',
       'Intel Celeron N3350', 'Intel Celeron N4020',
       'Intel Celeron N4100', 'Intel Celeron N4500',
       'Intel Celeron N5100', 'Intel Core Duo', 'Intel Core M3',
       'Intel Core i3', 'Intel Core i3 10th Gen.',
       'Intel Core i3 11th Gen.', 'Intel Core i3 12th Gen.',
       'Intel Core i3 4th Gen.', 'Intel Core i3 6th Gen.',
       'Intel Core i5', 'Intel Core i5 10th Gen.',
       'Intel Core i5 11th Gen.', 'Intel Core i5 12th Gen.',
       'Intel Core i5 13th Gen.', 'Intel Core i5 1st Gen.',
       'Intel Core i5 2nd Gen.', 'Intel Core i5 3rd Gen.',
       'Intel Core i5 4th Gen.', 'Intel Core i5 5th Gen.',
       'Intel Core i5 6th Gen.', 'Intel Core i5 7t

In [41]:
# converting i5 generations to i5
ebay_data_sample['Processor'] = ebay_data_sample['Processor'].replace(to_replace=['Intel Core i5 10th Gen.',
       'Intel Core i5 11th Gen.', 'Intel Core i5 12th Gen.',
       'Intel Core i5 13th Gen.', 'Intel Core i5 1st Gen.',
       'Intel Core i5 2nd Gen.', 'Intel Core i5 3rd Gen.',
       'Intel Core i5 4th Gen.', 'Intel Core i5 5th Gen.',
       'Intel Core i5 6th Gen.', 'Intel Core i5 7th Gen.',
       'Intel Core i5 8th Gen.', 'Intel Core i5 vPro 8th Gen'],
value='Intel Core i5')
# converting i3 generations to i3
ebay_data_sample['Processor'] = ebay_data_sample['Processor'].replace(to_replace=['Intel Core i3 10th Gen.',
       'Intel Core i3 11th Gen.', 'Intel Core i3 12th Gen.',
       'Intel Core i3 4th Gen.', 'Intel Core i3 6th Gen.'],
value='Intel Core i3')
# converting pentium types to pentium
ebay_data_sample['Processor'] = ebay_data_sample['Processor'].replace(to_replace=['Intel Pentium Gold',
       'Intel Pentium M', 'Intel Pentium N4200', 'Intel Pentium N5030',
       'Intel Pentium Silver N6000'],
value='Intel Pentium')
# converting celeron types to celeron
ebay_data_sample['Processor'] = ebay_data_sample['Processor'].replace(to_replace=['Intel Celeron N', 'Intel Celeron N2840', 'Intel Celeron N2940',
       'Intel Celeron N3050', 'Intel Celeron N3060',
       'Intel Celeron N3350', 'Intel Celeron N4020',
       'Intel Celeron N4100', 'Intel Celeron N4500',
       'Intel Celeron N5100'],
value='Intel Celeron')
# converting i7 generations to i7
ebay_data_sample['Processor'] = ebay_data_sample['Processor'].replace(to_replace=['Intel Core i7 10th Gen.',
       'Intel Core i7 11th Gen.', 'Intel Core i7 12th Gen.',
       'Intel Core i7 13th Gen.', 'Intel Core i7 2nd Gen.',
       'Intel Core i7 4th Gen.', 'Intel Core i7 5th Gen.',
       'Intel Core i7 6th Gen.', 'Intel Core i7 7th Gen.',
       'Intel Core i7 8th Gen.', 'Intel Core i7 9th Gen.'],
value='Intel Core i7')

Now we have narrowed down the processors to be more categorical. Hopefully this will make analysis easier if our statisticians are particularly interested in measuing price by intel processor type, for example.

In [42]:
# obtain every unique value in the 'Model' column
model_unique_values = ebay_data_sample['Model'].unique()
model_unique_values.sort()
model_unique_values

array(['14', '14 G6', '14-an013nr', '14A-NB0013DX', '15-ba018wm',
       '15-dy2091wm', '15-dy2795wm', '15-dy5131wm', '20NY-S3Y600',
       '47X83UA#ABA', '5410', '5580', '7290', '7530', '82R',
       'AN515-43-R0YM', 'ASUS VivoBook', 'Acer Aspire 1', 'Acer Aspire 3',
       'Acer C740', 'Acer Chromebook C720', 'Acer Chromebook C731',
       'Acer Chromebook Spin 511', 'Acer Chromebook Spin 713',
       'Acer Nitro 5', 'Acer Swift 1', 'Aerobook Pro', 'Alienware 15 R3',
       'C731T', 'C731T-C0X8', 'C732 Chromebook', 'CHUWI CoreBook X',
       'CHUWI FreeBook', 'CHUWI Herobook Pro', 'CHUWI LapBook Pro',
       'CHUWI Lapbook Pro', 'CHUWI MiniBook X', 'CHUWI UBOOK XPRO',
       'ChromeBook', 'Chromebook 11 C771-C4TM', 'Chromebook 11 G4',
       'Chromebook 11 G6 EE', 'Chromebook 11 G6 EE (3NU57UT#ABA)',
       'Chromebook 14 G5', 'Chromebook C738T-C44Z', 'Core i5',
       'Core i7-1165U', 'CoreBook X', 'DELL LATITUDE 5400', 'Dell',
       'Dell Chromebook', 'Dell Chromebook 13 -7310, i5

Ok, so we have tons of different laptop models. It will be too difficult to narrow these down into smaller categories, but I will try to look up uninformative terms like 'sl10w47275' online and try to match them to an existing or new model.

In [43]:
# clean up values
ebay_data_sample['Model'] = ebay_data_sample['Model'].replace({
    # searched this value and got the proper model denotation
    'sl10w47275': 'Lenovo ThinkPad E15',
    'ThinkPad E15':'Lenovo ThinkPad E15',
    # searched this value and got the proper model denotation
    '47X83UA#ABA':'14-dq0080nr',
    # normalize the capitalization
    'DELL LATITUDE 5400':'Dell Latitude 5440',
    'Chromebook 11 G6 EE (3NU57UT#ABA)':'Chromebook 11 G6 EE',
    'Precision 5760 Laptop':'Precision 5760',
    'Yoga 11e':'Lenovo Thinkpad Yoga 11e',
    'CHUWI Lapbook Pro':'CHUWI LapBook Pro',
    # assuming the '11th' does not have any significant value
    'hp elitebook 840 g8 11th':'Elitebook 840 G8',
    # adding the brand before the value
    'CoreBook X':'CHUWI CoreBook X',
    # normalize Generation denotation
    'Thinkpad L14 Gen 2':'Lenovo Thinkpad L14 G2',
    'asus q534u':'ASUS q534u',
    # remove the '2 in 1' because 7405 implies that, based on a search I made
    'Dell Inspiron 14 7405 2 in 1':'Dell Inspiron 14 7405',
    # the i5 is not relevant for the model, it is implied in the processor column
    'Dell Chromebook 13 -7310, i5':'Dell Chromebook 13 7310'})

# normalize Generation denotation 
ebay_data_sample['Model'] = ebay_data_sample['Model'].replace(to_replace=['Lenovo ThinkPad E14 2nd Gen',
       'Lenovo ThinkPad E14 Gen2'],
value='Lenovo ThinkPad E14 G2')


Because of how many different models there are, it's difficult to narrow down the grouping for this data. I've done some cleaning but I'm not sure how much else can be done because I don't want to classify any particular model as a different model by "cleaning" it. I'm not entirely satisfied with this column but I'm not sure proceeding will make any impact.

In [44]:
# looks like we have two columns left to evaluate: 'GPU' and 'Type'.
ebay_data_sample

,Item Number,Brand,Processor,Screen Size (in),Color,Ram Size (GB),SSD Capacity (GB),GPU,Processor Speed (GHz),Type,Resolution Size,Maximum Resolution,Model,OS,Hard Drive Capacity (GB),Storage Type,Condition,Price ($)
0,85264981001,Chuwi,Quad Core,14.0,Gray,8.0,1000.0,Intel Iris Plus Graphics 655,3.8,Notebook/Laptop,Large,2160 x 1440,CHUWI CoreBook X,Windows 11 Home,512,SSD,New,303.68
1,85264981003,Dell,Intel Core i5,14.0,Black,16.0,500.0,Intel HD Graphics,2.4,Notebook/Laptop,Medium,1920 x 1080,Dell Latitude E5470,Windows 10 Pro,500,SSD,Used,175.00
2,85264981014,Chuwi,Intel Celeron,14.1,Gray,8.0,256.0,"Intel UHD Graphics 600, 650MHz",2.8,Notebook/Laptop,Medium,1920 x 1080,CHUWI LapBook Pro,Windows 11 Home,256,SSD,New,151.99
3,85264981015,Chuwi,Intel Core i5,14.0,Gray,8.0,512.0,Intel Iris Plus Graphics 655,3.8,Notebook/Laptop,Large,2160 x 1440,CHUWI CoreBook X,Windows 10,512,SSD + HDD,New,243.99
4,85264981018,Dell,Intel Celeron,11.0,Black,4.0,128.0,Intel UHD Graphics 600,1.1,Notebook/Laptop,Small,1366 x 768,Dell Latitude 3189,Windows 10 Pro,128,SSD,Used,129.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,85264987126,HP,Intel Core i7,17.3,Silver,64.0,4000.0,Intel Iris Xe Graphics,1.7,Notebook/Laptop,Medium,1920 x 1080,HP 17-cn3097nr,Windows 11 Pro,1000,SSD + HDD,New,1280.00
457,85264987127,ASUS,AMD Ryzen 7,16.0,Blue,16.0,1000.0,AMD Radeon Graphics,4.4,Notebook/Laptop,Large,1920 x 1200,ASUS VivoBook,Windows 11 Home,1000,SSD,New,609.00
458,85264987136,Lenovo,AMD Ryzen 5,14.0,Black,16.0,256.0,AMD Radeon Graphics,2.3,Notebook/Laptop,Medium,1920 x 1080,Lenovo ThinkPad T14,Windows 10 Pro,256,SSD,New,499.99
459,85264987139,Lenovo,Intel Core i5,15.6,Grey,20.0,1000.0,Intel Iris Xe Graphics,4.2,Notebook/Laptop,Medium,1920 x 1080,Lenovo Ideapad 3i,Windows 11 Home,1000,SSD,New,599.00


In [45]:
# obtain every unique value in the 'GPU' column
gpu_unique_values = ebay_data_sample['GPU'].unique()
gpu_unique_values.sort()
gpu_unique_values

array(['2GB Nvidia Quadro M1000M', 'AMD R4 Graphics',
       'AMD Radeon 4GB GPU', 'AMD Radeon Graphics', 'AMD Radeon R4',
       'AMD Radeon R7 M350',
       'AMD Radeon RX 560X - 4GB / AMD Radeon Vega 8 Graphics',
       'AMD Radeon RX Vega 10', 'AMD Radeon Vega 8',
       'AMD Radeon™ Graphics', 'ATI Mobility Radeon X600', 'Integrated',
       'Integrated Graphics', 'Integrated Intel HD Graphics',
       'Integrated Intel UHD Graphics 630', 'Intel Arc 370M',
       'Intel Core i3-1005G1', 'Intel GMA', 'Intel HD',
       'Intel HD 520 Graphics', 'Intel HD Graphics',
       'Intel HD Graphics 3000', 'Intel HD Graphics 400',
       'Intel HD Graphics 4000', 'Intel HD Graphics 4600',
       'Intel HD Graphics 500', 'Intel HD Graphics 5000',
       'Intel HD Graphics 505', 'Intel HD Graphics 510',
       'Intel HD Graphics 515', 'Intel HD Graphics 520',
       'Intel HD Graphics 530', 'Intel HD Graphics 5300',
       'Intel HD Graphics 5500', 'Intel HD Graphics 6000',
       'Intel HD Gr

Like with 'Model', there are lots of different GPUs. I am going to normalize the brand names and get rid of extra terms like '6 GB GDDR6' and '(i7-6th Gen)'. I also am going to combine any "integrated" graphics into 'Intel HD Graphics', as the integrated term seems to simply be extra labeling.

In [46]:
# replace some unneccessary terms
ebay_data_sample['GPU'] = ebay_data_sample['GPU'].replace(to_replace=[
    r'\s*®',r'\s*™',r'\s*with 6GB GDDR5 dedicated memory',r'\s*250-750MHz',r'\s*2GB'], value='', regex=True)
ebay_data_sample['GPU'] = ebay_data_sample['GPU'].replace(to_replace=[
    r'\s*NVidia',r'\s*Nvidia',r'\s*nvidia'], value='NVIDIA', regex=True)

# continue replacing entries with better-looking entries
ebay_data_sample['GPU'] = ebay_data_sample['GPU'].replace({
# remove extra term '650 MHz'
    'Intel UHD Graphics 600, 650MHz': 'Intel UHD Graphics 600',
# rearrange for consistency
    'Intel HD 520 Graphics': 'Intel HD Graphics 520',
    'Integrated Graphics': 'Intel HD Graphics',
    'Integrated': 'Intel HD Graphics',
# remove extra term '4GB (i7-7th Gen),  M1000M (i7-6th Gen)'
    'NVIDIA Quadro M1200 4GB (i7-7th Gen),  M1000M (i7-6th Gen)': 'NVIDIA Quadro M1200',
# add brand
    'Quadro M620': 'NVIDIA Quadro M620',
    'AMD Radeon RX 560X - 4GB / AMD Radeon Vega 8 Graphics': 'AMD Radeon Vega 8',
    'Integrated Intel HD Graphics': 'Intel HD Graphics',
    'Intel Iris XE': 'Intel Iris Xe Graphics',
    'Integrated Intel UHD Graphics 630': 'Intel UHD Graphics 630',
    'NVIDIA RTX A1000,  6 GB GDDR6': 'NVIDIA RTX A1000',
    })

It seems like 'Intel Core i3-1005G1' is not a GPU. Thus, I am going to drop the entry containing this value.

In [47]:
ebay_data_sample = ebay_data_sample[ebay_data_sample['GPU'] != 'Intel Core i3-1005G1']

Finally, let us evaluate and clean the 'Type' column

In [48]:
type_unique_values = ebay_data_sample['Type'].unique()
type_unique_values.sort()
type_unique_values

array(['2 in 1 Laptop/Tablet', 'Chromebook', 'Computers',
       'Convertible 2-in-1 Laptop/Tablet', 'Gaming Laptop Computers',
       'Laptop', 'Laptop/Portable Workstation', 'NB', 'Netbook',
       'Notebook', 'Notebook/Laptop', 'Notebook/Laptop/Tablet',
       'Subnotebook/Ultraportable', 'Tablet', 'Tablet NoteBook',
       'Thinkpad', 'Ultrabook'], dtype=object)

In [49]:
ebay_data_sample['Type'] = ebay_data_sample['Type'].replace({
# rename to make more sense
    'Convertible 2-in-1 Laptop/Tablet': '2 in 1 Laptop/Tablet',
# a Tablet NoteBook is a type of Tablet, so I am going to reassign those values
    'Tablet NoteBook': 'Tablet',
# a portable workstation is another name for laptop.
    'Laptop/Portable Workstation':'Laptop',
    'Computers': 'Computer',
    'Gaming Laptop Computers':'Gaming Laptop Computer',
# assuming that NB denotes 'Notebook'
    'NB':'Notebook'
    })

C:\Users\aaron\AppData\Local\Temp\ipykernel_35080\840407415.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ebay_data_sample['Type'] = ebay_data_sample['Type'].replace({


In [50]:
ebay_data_sample.dtypes

Item Number                   int64
Brand                        object
Processor                    object
Screen Size (in)            float64
Color                        object
Ram Size (GB)               float64
SSD Capacity (GB)           float64
GPU                          object
Processor Speed (GHz)       float64
Type                         object
Resolution Size              object
Maximum Resolution           object
Model                        object
OS                           object
Hard Drive Capacity (GB)      int64
Storage Type                 object
Condition                    object
Price ($)                   float64
dtype: object

Now we have data that is better suited for interpretation. Brand, Screen Size, SSD Capacity, Processor Speed, and Hard Drive Capacity are now all numeric values that may correlate with price. Additionally, the brand, storage type, and resolution size attributes have few unique values, so they could be used as dummy variables or in box plots to see if price changes based on different values for those factors.

All of the columns I just mentioned would be most important to the business as they seek out what impacts price on their website.

In [51]:
# now we have cleaned all the data! time to drop any potential duplicates.
ebay_data_clean = ebay_data_sample.drop_duplicates()
ebay_data_clean

,Item Number,Brand,Processor,Screen Size (in),Color,Ram Size (GB),SSD Capacity (GB),GPU,Processor Speed (GHz),Type,Resolution Size,Maximum Resolution,Model,OS,Hard Drive Capacity (GB),Storage Type,Condition,Price ($)
0,85264981001,Chuwi,Quad Core,14.0,Gray,8.0,1000.0,Intel Iris Plus Graphics 655,3.8,Notebook/Laptop,Large,2160 x 1440,CHUWI CoreBook X,Windows 11 Home,512,SSD,New,303.68
1,85264981003,Dell,Intel Core i5,14.0,Black,16.0,500.0,Intel HD Graphics,2.4,Notebook/Laptop,Medium,1920 x 1080,Dell Latitude E5470,Windows 10 Pro,500,SSD,Used,175.00
2,85264981014,Chuwi,Intel Celeron,14.1,Gray,8.0,256.0,Intel UHD Graphics 600,2.8,Notebook/Laptop,Medium,1920 x 1080,CHUWI LapBook Pro,Windows 11 Home,256,SSD,New,151.99
3,85264981015,Chuwi,Intel Core i5,14.0,Gray,8.0,512.0,Intel Iris Plus Graphics 655,3.8,Notebook/Laptop,Large,2160 x 1440,CHUWI CoreBook X,Windows 10,512,SSD + HDD,New,243.99
4,85264981018,Dell,Intel Celeron,11.0,Black,4.0,128.0,Intel UHD Graphics 600,1.1,Notebook/Laptop,Small,1366 x 768,Dell Latitude 3189,Windows 10 Pro,128,SSD,Used,129.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,85264987126,HP,Intel Core i7,17.3,Silver,64.0,4000.0,Intel Iris Xe Graphics,1.7,Notebook/Laptop,Medium,1920 x 1080,HP 17-cn3097nr,Windows 11 Pro,1000,SSD + HDD,New,1280.00
457,85264987127,ASUS,AMD Ryzen 7,16.0,Blue,16.0,1000.0,AMD Radeon Graphics,4.4,Notebook/Laptop,Large,1920 x 1200,ASUS VivoBook,Windows 11 Home,1000,SSD,New,609.00
458,85264987136,Lenovo,AMD Ryzen 5,14.0,Black,16.0,256.0,AMD Radeon Graphics,2.3,Notebook/Laptop,Medium,1920 x 1080,Lenovo ThinkPad T14,Windows 10 Pro,256,SSD,New,499.99
459,85264987139,Lenovo,Intel Core i5,15.6,Grey,20.0,1000.0,Intel Iris Xe Graphics,4.2,Notebook/Laptop,Medium,1920 x 1080,Lenovo Ideapad 3i,Windows 11 Home,1000,SSD,New,599.00


In [52]:
# the old data, for comparison:
ebay_data.head(3)

,Item Number,Brand,Rating,Ratings Count,Processor,Screen Size,Manufacturer Color,Color,Ram Size,SSD Capacity,...,Maximum Resolution,Model,OS,Hard Drive Capacity,Country Region Of Manufacture,Storage Type,Features,Condition,Seller Note,Price
0,85264981001,CHUWI,NaN,NaN,Quad Core,14 in,NaN,Gray,8 GB,"1 Used, 1 support Max 1TB M.2 Extend",...,2160 x 1440,CoreBook X,Windows 11 Home,512 GB SSD,China,SSD (Solid State Drive),"Backlit Keyboard, Built-in Microphone, Built...","New: A brand-new, unused, unopened, undamaged ...",NaN,$303.68
1,85264981002,Dell,NaN,NaN,Intel Core i7 8th Gen.,14 in,NaN,Black,NaN,NaN,...,1920 x 1080,Dell Latitude 7490,Windows 11 Pro,2 TB,NaN,SSD (Solid State Drive),"Backlit Keyboard, Bluetooth, Built-in Micropho...",Very Good - RefurbishedThe item shows minimal ...,“AAA PCs is a Microsoft Authorized Refurbisher...,$399.99 to $634.99
2,85264981003,Dell,NaN,NaN,Intel Core i5-6300U,14 in,NaN,Black,16 GB,500 GB,...,1920 x 1080,Dell Latitude E5470,Windows 10 Pro,500 GB,China,SSD (Solid State Drive),"10/100 LAN Card, Backlit Keyboard, Bluetooth, ...",UsedAn item that has been used previously. The...,"“Well kept, fully functional, includes battery...",$175.00


In [53]:
ebay_data_clean.to_csv('ebay_data_clean.csv', index=False)